In [1]:
import pandas as pd
import numpy as np

Strompreis=pd.read_csv("Daten entsoe/Gro_handelspreise_202311010000_202502010000_Stunde.csv",na_values='-',header=0 ,sep=';', decimal=",", thousands=".")

In [2]:
Strompreis['Datum']=pd.to_datetime(Strompreis['Datum von'],format='%d.%m.%Y %H:%M')
Strompreis=Strompreis.sort_values('Datum',kind='stable')
Strompreis['Datum']=pd.to_datetime(Strompreis['Datum von'],format='%d.%m.%Y %H:%M')
Strompreis['Datum']=Strompreis['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
Strompreis['Date']=Strompreis['Datum'].dt.tz_convert('UTC')
Strompreis=Strompreis.set_index('Date')

In [3]:
datei="Daten entsoe/Physikalischer_Stromfluss_202311010000_202502010000_Viertelstunde.csv"
dtype={'Datum von':"string",'Datum bis':"string"
       ,'Nettoexport [MWh] Originalauflösungen':np.float64
       ,'Niederlande (Export) [MWh] Originalauflösungen':np.float64
       ,'Niederlande (Import) [MWh] Originalauflösungen':np.float64
       ,'Schweiz (Export) [MWh] Originalauflösungen':np.float64
       ,'Schweiz (Import) [MWh] Originalauflösungen':np.float64
       ,'Dänemark (Export) [MWh] Originalauflösungen':np.float64
       ,'Dänemark (Import) [MWh] Originalauflösungen':np.float64
       ,'Tschechien (Export) [MWh] Originalauflösungen':np.float64
       ,'Tschechien (Import) [MWh] Originalauflösungen':np.float64
       ,'Luxemburg (Export) [MWh] Originalauflösungen':np.float64
       ,'Luxemburg (Import) [MWh] Originalauflösungen':np.float64
       ,'Schweden (Export) [MWh] Originalauflösungen':np.float64
       ,'Schweden (Import) [MWh] Originalauflösungen':np.float64
       ,'Österreich (Export) [MWh] Originalauflösungen':np.float64
       ,'Österreich (Import) [MWh] Originalauflösungen':np.float64
       ,'Frankreich (Export) [MWh] Originalauflösungen':np.float64
       ,'Frankreich (Import) [MWh] Originalauflösungen':np.float64
       ,'Polen (Export) [MWh] Originalauflösungen':np.float64
       ,'Polen (Import) [MWh] Originalauflösungen':np.float64
       ,'Norwegen (Export) [MWh] Originalauflösungen':np.float64
       ,'Norwegen (Import) [MWh] Originalauflösungen':np.float64
       ,'Belgien (Export) [MWh] Originalauflösungen':np.float64
       ,'Belgien (Import) [MWh] Originalauflösungen':np.float64}


#Stromfluss=pd.read_csv("Daten entsoe/Physikalischer_Stromfluss_202311010000_202502010000_Viertelstunde.csv",sep=';', decimal=",", thousands=".",dtype=dtype,on_bad_lines='skip')
Stromfluss=pd.read_csv(datei,sep=';', decimal=",", thousands=".",header=0,na_values='-',on_bad_lines='skip')

In [4]:
Stromfluss['Datum']=pd.to_datetime(Stromfluss['Datum von'],format='%d.%m.%Y %H:%M')
#Stromfluss=Stromfluss.sort_values('Datum',kind='stable')
Stromfluss['Datum']=pd.to_datetime(Stromfluss['Datum von'],format='%d.%m.%Y %H:%M')
Stromfluss['Datum']=Stromfluss['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
Stromfluss['Date']=Stromfluss['Datum'].dt.tz_convert('UTC')
Stromfluss=Stromfluss[Stromfluss['Date'].dt.year==2024]
Stromfluss=Stromfluss.set_index('Date').fillna(0)
pm_cat=Stromfluss['Nettoexport [MWh] Originalauflösungen']
Stromfluss['+-']=(np.where(pm_cat<0,'0: Import','1: Export'))
Strompreis=Strompreis.reindex(Stromfluss.index).ffill()
#Stromfluss.rename(columns={'Nettoexport [MWh] Originalauflösungen':'Nettoexport [TWh]'})
Stromfluss['Preis']=Strompreis['Deutschland/Luxemburg [€/MWh] Originalauflösungen']
Stromfluss['Nettoexport [TWh]']=Stromfluss['Nettoexport [MWh] Originalauflösungen']/1000000
Stromfluss['Wert Mio €']=Stromfluss['Preis']*Stromfluss['Nettoexport [MWh] Originalauflösungen']/1000000
#Stromfluss=Stromfluss.set_index('Date')

In [22]:
# funktioniert: 
#Stromfluss.groupby([Stromfluss.index.month,Stromfluss['+-']])[['Nettoexport [TWh]','Wert Mio €']].agg(['sum'])
df=Stromfluss.reset_index()
df['Monat']=df['Date'].dt.month
df['Jahr']=df['Date'].dt.year
Strom=pd.pivot_table(df,index='Monat', columns=['+-'],values=['Wert Mio €','Nettoexport [TWh]'], aggfunc='sum',margins=True,margins_name='2024')

In [23]:
#Strom['Durchschnittspreis [€/MWh]','0: Import']=Strom['Wert Mio €']['0: Import']/Strom['Nettoexport [TWh]']['0: Import']
#Strom['Durchschnittspreis [€/MWh]','1: Export']=Strom['Wert Mio €']['1: Export']/Strom['Nettoexport [TWh]']['1: Export']
Strom=Strom.join(Strom[['Wert Mio €']].div(Strom['Nettoexport [TWh]']).rename(columns={'Wert Mio €':'Durchschnittspreis [€/MWh]'}))
Strom=Strom.drop(columns=('Durchschnittspreis [€/MWh]','2024'), axis=1)

In [24]:
Strom

Nettoexport [TWh]                         Wert Mio €              \
+-            0: Import  1: Export       2024    0: Import   1: Export   
Monat                                                                    
1             -1.134009   2.727416   1.593407  -109.468334  153.401560   
2             -1.703808   1.916092   0.212284  -126.664725   72.941809   
3             -2.723841   1.008719  -1.715123  -214.313978   33.724300   
4             -3.103580   0.779169  -2.324411  -272.950747    1.423913   
5             -4.105454   0.678305  -3.427149  -367.218140  -11.308225   
6             -4.055072   0.492438  -3.562634  -378.533584   -3.923504   
7             -4.767093   0.476923  -4.290170  -419.617906   -5.529356   
8             -4.784844   0.505738  -4.279106  -492.243474   -5.031528   
9             -3.162973   0.553136  -2.609837  -336.588938    5.009270   
10            -3.740583   0.348937  -3.391646  -377.071172    4.122459   
11            -2.707387   1.521734  -1.185653  -399.674884  115.197716   
12            -1.958986   2.147534   0.188549  -405.004888  129.183802   
2024         -37.947629  13.156141 -24.791488 -3899.350769  489.212216   

                   Durchschnittspreis [€/MWh]             
+-            2024                  0: Import  1: Export  
Monat                                                     
1        43.933226                  96.532177  56.244294  
2       -53.722916                  74.342124  38.068005  
3      -180.589678                  78.680782  33.432816  
4      -271.526835                  87.947056   1.827475  
5      -378.526365                  89.446414 -16.671293  
6      -382.457088                  93.348178  -7.967512  
7      -425.147262                  88.023860 -11.593819  
8      -497.275003                 102.875564  -9.948883  
9      -331.579668                 106.415369   9.056127  
10     -372.948712                 100.805462  11.814328  
11     -284.477169                 147.623847  75.701611  
12     -275.821085                 206.742131  60.154471  
2024  -3410.138554                 102.756109  37.185084

In [25]:
datei='Daten entsoe/Realisierte_Erzeugung_202311010000_202502010000_Viertelstunde.csv'
Stromerzeugung=pd.read_csv(datei,sep=';', decimal=",", thousands=".",header=0,na_values='-',on_bad_lines='skip')

In [27]:
Stromerzeugung['Datum']=pd.to_datetime(Stromerzeugung['Datum von'],format='%d.%m.%Y %H:%M')
Stromerzeugung['Datum']=Stromerzeugung['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
Stromerzeugung['Date']=Stromerzeugung['Datum'].dt.tz_convert('UTC')
Stromerzeugung=Stromerzeugung[Stromerzeugung['Date'].dt.year==2024]
Stromerzeugung=Stromerzeugung.set_index('Date').fillna(0)

In [31]:
Stromerzeugung=Stromerzeugung.rename(columns={'Wind Offshore [MWh] Originalauflösungen':'Wind Off','Wind Onshore [MWh] Originalauflösungen':'Wind On','Photovoltaik [MWh] Originalauflösungen':'PV'})

In [65]:
pv=Stromerzeugung['PV']/1000000
divi=3
x=sum(pv)
y=sum(np.where(pv>max(pv)/divi,max(pv)/divi,pv))
z=y/x
z,y,x

(0.6903129637218921, 43.67912374999577, 63.2743785)

In [59]:
sum(np.where(pv>max(pv)/2,max(pv)/2,pv))

54.70061799999507

In [55]:
max(Stromerzeugung['PV'])

11753.0